# KPI and Threshold Criteria Analysis

## Import simulation run data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from model.plot_utils import *
import seaborn as sns
import statsmodels.api as sm
import pickle
from pathlib import Path
import os
%matplotlib inline

In [ ]:
# The 'root_dir' directory here should be the local version of 'hydra/hydra_multi_class'
root_dir = os.getcwd()

In [ ]:
results = pd.read_pickle ('C:/Users/paruc/Documents/Github/hydra/hydra_multi_class/experiment_block1_experiments20210429.pkl')

In [ ]:
experiment_path = root_dir + '/experiment_block1_experiments_sigma2_20210511'

In [ ]:
# Get experiment results files stored in 'experiments' subdirectory
paths = sorted(Path(experiment_path).iterdir(), key=os.path.getmtime)

In [ ]:
# Open the most recent experiment--alternatively, substitute filename for desired experiment results file
with open(paths[-1], 'rb') as f:
    config_ids, experiments = pickle.load(f)

In [ ]:
experiments.columns

In [ ]:
# Representative fan plot of experimental results (sanity check desired experiment was loaded)
param_fan_plot3(experiments, config_ids, 'a','Q')

## Experiment pre-processing

### Define Slippage and Impermanent Loss calculations


#### Slippage

Slippage is calculated in two different, but related ways: the first is as an elasticity, by measuring the percentage change in the price following a trade with respect to the trade size as a percentage of the pool reserve. The second is as the percentage difference between the effective (actual trade) price and the spot price before the trade, which measures trader expectations on the execution price.

#### Impermanent Loss

Impermanent Loss (IL) is computed as the difference between the value of an amount of a single asset provided  as liquidity by an LP when exiting the pool, and the value of the intial single asset amount held outside of the pool. 


In [ ]:
def slippage(assets, rdf, market):
    
    slippage = []
    elasticity = []
    res_in = []
    res_out = []
    trans_in = []
    trans_out = []
    p_out_for_in = []
    
    # Assign ids for risk assets IN and OUT for Hydra
    asset_in_ids = rdf.asset_random_choice
    asset_out_ids = pd.Series([x for y in asset_in_ids for x in assets if x != y  ])
    
    if market == 'hydra': 
        
        for t in range(0,len(asset_in_ids)-1):
            
            reserve_asset_in_prev = rdf.pool[t].pool[asset_in_ids[t]]['R']
            reserve_asset_in = rdf.pool[t+1].pool[asset_in_ids[t]]['R']
            reserve_asset_out = rdf.pool[t+1].pool[asset_out_ids[t]]['R']
            reserve_asset_out_prev = rdf.pool[t].pool[asset_out_ids[t]]['R']
            price_asset_in = rdf.pool[t+1].pool[asset_in_ids[t]]['P']
            price_asset_in_prev = rdf.pool[t].pool[asset_in_ids[t]]['P']
            price_asset_out = rdf.pool[t+1].pool[asset_in_ids[t]]['P']
            price_asset_out_prev = rdf.pool[t].pool[asset_out_ids[t]]['P']
            
            transactions_in = rdf.trade_random_size[t]
            transactions_out = -(reserve_asset_out - reserve_asset_out_prev)

            # If transactions_out or transactions_in is zero, was not a swap/trade event; return n.a.
            if transactions_out == 0 or transactions_in == 0:
                elasticity.append(np.nan)
                slippage.append(np.nan)
            else:
                # Compute percent change in reserve
                reserve_in_pct_change = transactions_in / reserve_asset_in_prev
                reserve_out_pct_change = transactions_out / reserve_asset_out_prev

                # Compute percent change in price (OUT per IN)
                price_out_for_in = price_asset_out / price_asset_in
                price_out_for_in_prev = price_asset_out_prev / price_asset_in_prev
                price_pct_change = (price_out_for_in - price_out_for_in_prev) / price_out_for_in_prev

                # Slippage calculation #1: elasticity of price with respect to transactions size
                elasticity.append(price_pct_change / reserve_in_pct_change)
                # Slippage calculation #2: percentage difference between effective and spot price

                slippage.append(((transactions_in/transactions_out) - price_out_for_in_prev) 
                                / price_out_for_in_prev)
            
            res_in.append(reserve_asset_in)
            res_out.append(reserve_asset_out)
            trans_in.append(transactions_in)
            trans_out.append(transactions_out)
            p_out_for_in.append(price_out_for_in)

        # Extract time series - clunky, refactor into comprehensions for performance
        #reserve_asset_in_dict = {}
        #reserve_asset_out_dict = {}
        #asset_price_in_dict = {}  # Note: in terms of HDX as numeraire
        #asset_price_out_dict = {} # Note: in terms of HDX as numeraire
        #asset_weight_in_dict = {}
        #asset_weight_out_dict = {}
        #for t in rdf.pool.keys():
        #    reserve_asset_in_dict[t] = rdf.pool[t].pool[asset_in_id]['R']
        #    reserve_asset_out_dict[t] = rdf.pool[t].pool[asset_out_id]['R']
        #    asset_price_in_dict[t] = rdf.pool[t].pool[asset_in_id]['P']
        #    asset_price_out_dict[t] = rdf.pool[t].pool[asset_out_id]['P']
        #    asset_weight_in_dict[t] = rdf.pool[t].pool[asset_in_id]['W']
        #    asset_weight_out_dict[t] = rdf.pool[t].pool[asset_out_id]['W']
        #reserve_asset_in = pd.Series(reserve_asset_in_dict)
        #reserve_asset_out = pd.Series(reserve_asset_out_dict)
        #asset_price_in = pd.Series(asset_price_in_dict)
        #asset_price_out = pd.Series(asset_price_out_dict)
        #asset_weight_in = pd.Series(asset_weight_in_dict)
        #asset_weight_out = pd.Series(asset_weight_out_dict)
        
        # Compute price of OUT asset in terms of IN asset
        #price_out_for_in = asset_price_out / asset_price_in

    elif market == 'uni':
        
        for t in range(0,len(asset_in_ids)-1):
            asset_in_id = 'UNI_' + str(asset_in_ids[t]) + str(asset_out_ids[t])
            asset_out_id = 'UNI_' + str(asset_out_ids[t]) + str(asset_in_ids[t])
            price_id = 'UNI_P_ij'

            reserve_asset_in = rdf[asset_in_id][t+1]
            reserve_asset_in_prev = rdf[asset_in_id][t]
            reserve_asset_out = rdf[asset_out_id][t+1]
            reserve_asset_out_prev = rdf[asset_out_id][t]
            
            transactions_in = rdf.trade_random_size[t]
            transactions_out = -(reserve_asset_out - reserve_asset_out_prev)

            # If transactions_out or transactions_in is zero, was not a swap/trade event; return n.a.
            if transactions_out == 0 or transactions_in == 0:
                elasticity.append(np.nan)
                slippage.append(np.nan)
            else:
                # Compute percent change in reserve
                reserve_in_pct_change = transactions_in / reserve_asset_in_prev
                reserve_out_pct_change = transactions_out / reserve_asset_out_prev
                # Compute percent change in price (OUT per IN)
                if asset_in_ids[t] == 'i':
                    price_out_for_in = rdf[price_id][t+1]
                    price_out_for_in_prev = rdf[price_id][t]
                elif asset_in_ids[t] == 'j':
                    price_out_for_in = 1 / rdf[price_id][t+1]
                    price_out_for_in_prev = 1 / rdf[price_id][t]
                price_pct_change = (price_out_for_in - price_out_for_in_prev) / price_out_for_in_prev

                # Slippage calculation #1: elasticity of price with respect to transactions size
                elasticity.append(price_pct_change / reserve_in_pct_change)

                # Slippage calculation #2: percentage difference between effective and spot price
                slippage.append(((transactions_in/transactions_out) - price_out_for_in_prev) 
                                / price_out_for_in_prev)
            
            res_in.append(reserve_asset_in)
            res_out.append(reserve_asset_out)
            trans_in.append(transactions_in)
            trans_out.append(transactions_out)
            p_out_for_in.append(price_out_for_in)
        
        #asset_in_id = 'UNI_R' + str(asset_in)
        #asset_out_id = 'UNI_R' + str(asset_out)
        #asset_in_swap_id = 'UNI_' + str(asset_in) + str(asset_out)
        #asset_out_swap_id = 'UNI_' + str(asset_out) + str(asset_in)
        #asset_price_in = 'UNI_P_RQ' + str(asset_in) # Note: in terms of HDX as numeraire
        #asset_price_out = 'UNI_P_RQ' + str(asset_out) # Note: in terms of HDX as numeraire
        #asset_price_swap = 'UNI_P_ij'
        
        #reserve_asset_in = rdf[asset_in_id]
        #reserve_asset_out = rdf[asset_out_id]
        #asset_price_in = rdf[asset_price_in]
        #asset_price_out = rdf[asset_price_out]
        #reserve_asset_in = rdf[asset_in_swap_id]
        #reserve_asset_out = rdf[asset_out_swap_id]
        
        #if asset_in == 'i':
        #    price_out_for_in = rdf[asset_price_swap]
        #elif asset_in == 'j':
        #    price_out_for_in = 1 / rdf[asset_price_swap]
        
        # 8 May 2021: column 'UNI_Rx' keeps track of Q <-> asset and add/remove Liquidity
        # columns 'UNI_ij' & 'UNI_ji' keep track of asset i <-> asset j swap/trades
        # To get full reserve balance effects, need to diff and update 'UNI_Rx' columns
        # reserve_asset_in = pd.Series([rdf[asset_in_id][0]])
        # reserve_asset_out = pd.Series([rdf[asset_out_id][0]])
        #for t in range(1,len(rdf[asset_in_id].diff())):
        #    add_in = pd.Series([reserve_asset_in[t-1] + rdf[asset_in_id].diff()[t] + 
        #                           rdf[asset_in_swap_id].diff()[t]])
        #    add_out = pd.Series([reserve_asset_out[t-1] + rdf[asset_out_id].diff()[t] + 
        #                           rdf[asset_out_swap_id].diff()[t]])
        #    reserve_asset_in = reserve_asset_in.append(add_in, ignore_index = True)
        #    reserve_asset_out = reserve_asset_out.append(add_out, ignore_index = True)
    
    return {
        'reserve_asset_in' : pd.Series(res_in),
        'reserve_asset_out' : pd.Series(res_out),
        'transactions_in' : pd.Series(trans_in), 
        'transactions_out' : pd.Series(trans_out),
        'price_out_for_in' : pd.Series(p_out_for_in),
        'elasticity' : pd.Series(elasticity), 
        'slippage' : pd.Series(slippage)
    }

In [ ]:
def impermanent_loss(agent_id, asset_id, liquidity_added, time_entered, time_exited, rdf, market):
    
    # Set ids for asset that the LP has added
    agent_asset_id = 'r_' + str(asset_id) + '_out'
    share_id = 's_' + str(asset_id)

    # Set amount added by LP during addLiquidity event 
    # (usually set as an initial condition of the simulation)
    liquidity_added = liquidity_added

    # Set timestamp of addLiquidity event for LP
    time_liquidity_in = time_entered # Note: this is different for Uniswap agent than Hydra agent LP!

    # Set timestamp of removeLiquidity event for LP
    time_liquidity_out = time_exited
    
    # Build cumulative transactions IN amount (assumes fee eventually assessed to IN asset)
    transactions_in_cumulative = rdf.trade_random_size.cumsum()[:time_liquidity_out]
    transactions_in_cumulative.name = "transactions_in_cumulative"

    if market == 'hydra':
        
        # Get share awarded to LP from addLiquidity event
        share_rewarded = ( rdf['hydra_agents'][time_liquidity_in].iloc[agent_id][share_id] - 
                             rdf['hydra_agents'][time_liquidity_in - 1].iloc[agent_id][share_id] )

        # Compute impermanent loss over entire time series
        Ri_over_Wi = []
        Price = []
        # Build loop computing IL for every timestep
        for t in rdf['hydra_agents'].keys():
            if t <= time_liquidity_out:
                Ri_over_Wi.append(rdf.pool[t].pool[asset_id]['R'] / rdf.pool[t].pool[asset_id]['W'])
                Price.append(rdf.pool[t].pool[asset_id]['P'])
        Price = pd.Series(Price)
        Ri_over_Wi = pd.Series(Ri_over_Wi)
        Wq_over_Sq = rdf['Wq'][:time_liquidity_out] / rdf['Sq'][:time_liquidity_out]
        liquidity_removed = Wq_over_Sq.reset_index(drop=True) * Ri_over_Wi.reset_index(drop = True) * share_rewarded
        hold_value = liquidity_added * Price
        pool_value = liquidity_removed * Price
        
        # Get reserve time series for asset the LP has added
        reserve_asset_in = []
        for t in rdf.pool.keys():
            if t<= time_liquidity_out:
                reserve_asset_in.append(rdf.pool[t].pool[str(asset_id)]['R'])
        reserve_asset_in = pd.Series(reserve_asset_in, name='reserve_asset_in')
        
    elif market == 'uni':
        
        asset_reserve_key = 'UNI_R' + str(asset_id)
        asset_share_key = 'UNI_S' + str(asset_id)
        asset_price_key = 'UNI_P_RQ' + str(asset_id)
        asset_in_id = 'UNI_R' + str(asset_id)
        if asset_id == 'i':
            asset_out_id = 'j'
        elif asset_id == 'j':
            asset_out_id = 'i'
        asset_in_swap_id = 'UNI_' + str(asset_id) + str(asset_out_id)

        # Get share awarded to LP from addLiquidity event
        share_rewarded = ( rdf['uni_agents'][time_liquidity_in].iloc[agent_id][share_id] - 
                             rdf['uni_agents'][time_liquidity_in - 1].iloc[agent_id][share_id] )

        # Compute impermanent loss over entire time series

        liquidity_removed = (rdf[asset_reserve_key][:time_liquidity_out+1] / 
                               rdf[asset_share_key][:time_liquidity_out+1]) * share_rewarded
        pool_value = liquidity_removed * rdf[asset_price_key][:time_liquidity_out+1] 
        hold_value = liquidity_added * rdf[asset_price_key][:time_liquidity_out+1]
        
        # Get reserve time series for asset the LP has added
        # 8 May 2021: column 'UNI_Rx' keeps track of Q <-> asset and add/remove Liquidity
        # columns 'UNI_ij' & 'UNI_ji' keep track of asset i <-> asset j swap/trades
        # To get full reserve balance effects, need to diff and update 'UNI_Rx' columns
        #reserve_asset_out = pd.Series([rdf[asset_out_id][0]])
        reserve_asset_in = [rdf[asset_in_id][0]]
        for t in range(1,len(rdf[asset_in_id].diff())):
            if t <= time_liquidity_out:
                add_in = reserve_asset_in[t-1] + rdf[asset_in_id].diff()[t] + rdf[asset_in_swap_id].diff()[t]
            #add_out = pd.Series([reserve_asset_out[t-1] + rdf[asset_out_id].diff()[t] + 
            #                       rdf[asset_out_swap_id].diff()[t]])
                reserve_asset_in.append(add_in)
        reserve_asset_in = pd.Series(reserve_asset_in, name='reserve_asset_in')
            #reserve_asset_out = reserve_asset_out.append(add_out, ignore_index = True)
    
    IL = (pool_value / hold_value) - 1
    return {
        'pool_value' : pool_value,
        'hold_value' : hold_value,
        'impermanent_loss' : IL,
        'liquidity_removed' : liquidity_removed,
        'transactions_in_cumulative' : transactions_in_cumulative,
        'reserve_asset_in' : reserve_asset_in
    }

#### For debugging only: create one representative 'rdf' result DataFrame for subset 0 and simulation 0

In [ ]:
subset_array = experiments['subset'].unique()
MC_simulation_array = experiments['simulation'].unique()
experiment_by_subset = experiments.sort_values(by=['subset']).reset_index(drop=True)
sub_ex = experiment_by_subset[experiment_by_subset['subset']==0].copy()
rdf = sub_ex[sub_ex['simulation']==0].sort_values(by=['timestep']).reset_index(drop=True).copy()


## Compute Slippage KPI time series across subsets and simulations

In [ ]:
sl_kpis = {}
assets = ['i', 'j']
for subset in subset_array:
    kpi_subset = { 'Hydra' : {}, 'UNI' : {} }
    sub_experiments = experiment_by_subset[experiment_by_subset['subset']==subset].copy()
    for simulation in MC_simulation_array:
        sub_monte_carlo = sub_experiments[sub_experiments['simulation'] == simulation]
        rdf = sub_monte_carlo.sort_values(by=['timestep']).reset_index(drop=True).copy()
        # the following snippet is for future reference if fees are analyzed
        # requires that the 'config_ids' variable be saved from experiments
        # *****
        #config_rdf = [x for x in config_ids if x['simulation_id'] == simulation and 
        #              x['subset_id'] == subset][0]
        #fee = 1 - config_rdf['M']['fee_numerator']/config_rdf['M']['fee_denominator']
        # *****
        print("***Slippage calc for sim ", simulation, " of subset ", subset, "***")
        kpi_subset['Hydra'].update({simulation : slippage(assets, rdf, 'hydra')})
        kpi_subset['UNI'].update({simulation : slippage(assets, rdf, 'uni')})
    sl_kpis.update({subset: kpi_subset})

## Compute Impermanent Loss KPI time series across subsets and simulations

In [ ]:
il_kpis = {}

# The following information should ideally be read in from e.g. 'config_ids',
# so that minimal intervention is required. Here, these are manually entered for
# the experiment being analyzed.
# **********
asset_in = 'i'
lp_agent_number = 2
liquidity_added = 50000
time_entered_hydra = 10
time_entered_uni = 10
time_exited = 90
# **********

for subset in subset_array:
    kpi_subset = { 'Hydra' : {}, 'UNI' : {} }
    sub_experiments = experiment_by_subset[experiment_by_subset['subset']==subset].copy()
    for simulation in MC_simulation_array:
        sub_monte_carlo = sub_experiments[sub_experiments['simulation'] == simulation]
        rdf = sub_monte_carlo.sort_values(by=['timestep']).reset_index(drop=True).copy()
        print("***IL calc for sim ", simulation, " of subset ", subset, "***")
        kpi_subset['Hydra'].update({simulation : impermanent_loss(
            lp_agent_number, asset_in, liquidity_added, 
            time_entered_hydra, time_exited, 
            rdf, 'hydra')})
        kpi_subset['UNI'].update({simulation : impermanent_loss(
            lp_agent_number, asset_in, liquidity_added, 
            time_entered_uni, time_exited, 
            rdf, 'uni')})
    il_kpis.update({subset: kpi_subset})

## (In progress) Plot KPI time series as fan plots, one per sweep value

In [ ]:
for subset in subset_array:
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(15,7))
    ax1.plot([sl_kpis[subset]['Hydra'][x]['slippage'] for x in sl_kpis[subset]['Hydra']])
    ax2.plot([sl_kpis[subset]['UNI'][x]['slippage'] for x in sl_kpis[subset]['Hydra']])

## Compute Threshold KPIs (regression results) from KPI time series
(Note that printing of regression results can be enabled/disabled using the PRINT constant)

In [ ]:
PRINT = True

coeffs = {
    'slippage' : [],
    'elasticity' : [],
    'impermanent_loss' : []
}

kpi_threshold_values = {
    'Hydra' : coeffs.copy(),
    'UNI' : coeffs.copy()
}

kpi_thresholds = {}

for subset in subset_array:
    kpi_threshold_values = {
        'Hydra' : coeffs.copy(),
        'UNI' : coeffs.copy()
    }
    for simulation in MC_simulation_array:
        for market in ['Hydra', 'UNI']:
            for measure in ['elasticity', 'slippage', 'impermanent_loss']: 
            #for measure in ['impermanent_loss']: 
                if measure in ['elasticity']:
                    depvar = sl_kpis[subset][market][simulation][measure]
                    indepvar = sl_kpis[subset][market][simulation]['transactions_out']
                    xname = ['Transactions Size']
                elif measure in ['slippage']:
                    depvar = sl_kpis[subset][market][simulation][measure]
                    indepvar = sl_kpis[subset][market][simulation]['transactions_out']
                    indepvar = sm.add_constant(indepvar)
                    xname = ['Constant', 'Transactions Size']
                elif measure in ['impermanent_loss']:
                    depvar = il_kpis[subset][market][simulation][measure]
                    indepvar = pd.concat([il_kpis[subset][market][simulation]['transactions_in_cumulative'],
                                        il_kpis[subset][market][simulation]['reserve_asset_in']], axis=1)
                    indepvar = sm.add_constant(indepvar)
                    xname = ['Constant', 'Transactions Size', 'Balance Size']
                if PRINT:
                    print('********************')
                    print('Market: ', market, '; measure: ', measure)
                    print('********************')
        
                try:
                    model = sm.OLS(depvar,indepvar, missing = 'drop')
                    results = model.fit()
                    if PRINT:
                        print(results.summary(xname=xname))
                    kpi_threshold_values[market][measure].append((results.params, results.pvalues))
                except ValueError as e:
                    print('Market: ', market, ' with measure: ', measure, '; error: ', e)
    kpi_thresholds.update({subset : kpi_threshold_values})

## KPI Threshold Assessment

Recall threshold criteria (as of 7 May 2021) are:
1. The estimated coefficients of the Hydra elasticity regressions should be less than one more than 80% of the MC runs.
2. The estimated coefficients of the Hydra slippage regressions should be less than the associated coefficient values of the Uniswap slippage regressions more than 80% of the MC runs, when transactions are the same across both markets.
3. The estimated constant and trade/swap transactions size coefficients of the Hydra impermanent loss regressions should be less than the associated coefficient values of the Uniswap impermanent loss regressions more than 80% of the MC runs, when transactions are the same across both markets.
4. The estimated reserve balance coefficient of the Hydra impermanent loss regressions should be statistically no different from zero for at least 80% of the MC runs.

In [ ]:
# TC1
TC1 = {}
for subset in subset_array:
    mar = {}
    kpi_threshold_values = kpi_thresholds[subset]
    for market in ['Hydra', 'UNI']:
        elasticity_coeffs = kpi_threshold_values[market]['elasticity']
        satisfy = len([x for x in elasticity_coeffs[0] if x.item() < 1])
        fraction = satisfy/len(elasticity_coeffs)
        mar.update({market : fraction})
    TC1.update({ subset : mar })
print("TC1: ", TC1)

In [ ]:
# TC2
TC2 = {}
for subset in subset_array:
    kpi_threshold_values = kpi_thresholds[subset]
    slippage_coeffs_h = kpi_threshold_values['Hydra']['slippage']
    slippage_coeffs_u = kpi_threshold_values['UNI']['slippage']
    satisfy = len([x for i, x in enumerate(slippage_coeffs_h[0]) if
                          x['const'] < slippage_coeffs_u[0][i]['const'] and
                          x[0] < slippage_coeffs_u[0][i][0]])
    fraction = satisfy/len(slippage_coeffs_h)
    TC2.update({subset: fraction})
print("TC2: ", TC2)

In [ ]:
# TC3, TC4
TC3 = {}; TC4 = {}
two_sided_significance = 0.05
for subset in subset_array:
    kpi_threshold_values = kpi_thresholds[subset]
    il_coeffs_h = kpi_threshold_values['Hydra']['impermanent_loss']
    il_coeffs_u = kpi_threshold_values['UNI']['impermanent_loss']
    satisfyTC3 = len([x for i, x in enumerate(il_coeffs_h[0]) if
                          x['const'] < il_coeffs_u[0][i]['const'] and
                          x['transactions_in_cumulative'] < il_coeffs_u[0][i]['transactions_in_cumulative']])
    satisfyTC4 = len([x for i, x in enumerate(il_coeffs_h[1]) if
                          x['reserve_asset_in'] < two_sided_significance])
    fractionTC3 = satisfyTC3/len(il_coeffs_h)
    fractionTC4 = satisfyTC4/len(il_coeffs_h)
    TC3.update({subset: fractionTC3})
    TC4.update({subset: fractionTC4})
print("TC3: ", TC3)
print("TC4: ", TC4)

## Possible future refinement: remove outliers from regresssion results using outlier test
Outlier test from: [StackOverflow](https://stackoverflow.com/questions/11882393/matplotlib-disregard-outliers-when-plotting)

In [ ]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh